### This is a Python client to ESP8266 WebREPL 
* Python Ver : 3.7
* ESP8266 Firmware: esp8266-20210902-v1.17.bin
* Websocket Frame: https://www.rfc-editor.org/rfc/rfc6455.html#section-5.2
* Masking Algorithm: https://datatracker.ietf.org/doc/html/rfc6455#section-5.3

In [1]:
import struct, socket 
from random import randint

class WEBREPL:
    def upgrade_http_to_websocket(self):
        frw = self.s.makefile("rwb", 0)
        frw.write(b"""GET / HTTP/1.1\r\nHost: echo.websocket.org\r\nConnection: Upgrade\r\nUpgrade: websocket\r\nSec-WebSocket-Key: foo\r\n\r\n""")  
        while True:
            l = frw.readline() 
            if l == b'\r\n': break  
            # Response = '''HTTP/1.1 101 Switching Protocols\r\nUpgrade: websocket\r\nConnection: Upgrade\r\nSec-WebSocket-Accept: BM0S0+ghftShuFVHQATK/DBiJq8=\r\n\r'''
    
        buf = b''
        while 1:
            c = self.s.recv(1)
            buf += c
            if c == b':': 
                self.s.recv(1)
                break 
    
    def send(self, payload): 
        l = len(payload)
        if l > 125: raise 
        key = [randint(0, 255),randint(0, 255),randint(0, 255),randint(0, 255)]
        masked_payload = [ord(c)^key[i%4] for i, c in enumerate(payload) ]
        frame = struct.pack(f">{6+l}B", 0b10000001, 0b10000000 | l, *key,  *masked_payload) 
        self.s.send(frame)
        
    def recv(self): 
        buf = b''
        while True:
            _ = self.s.recv(1) 
            assert _ == b'\x81' # Fin + reserve + OpCode
            l = ord(self.s.recv(1))
            buf += self.s.recv(l)    
            if buf[-4:]==b'>>> ':  
                return buf.decode()   
        
    def __init__(self, host='192.168.1.1', port=8266, password='123456'): 
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        self.s.connect((host, port))
        self.upgrade_http_to_websocket()
        self.send(password) 
        self.send('\r')
        print(self.recv())
        
    def close(self):
        self.s.send(struct.pack(f">6B", 0x88, 0x80, 0x00, 0x00, 0x00, 0x00))

In [2]:
webrepl = WEBREPL(host='192.168.x.129', password='123456')


WebREPL connected
>>> 


In [3]:
a = '''
import time  
from machine import Pin, PWM 
PIN_MOSFET = 4
PIN_Servo = 5 
PIN_BUILTIN_LED = 2
p0 = Pin(PIN_MOSFET, Pin.OUT, value=0)  
p2 = Pin(PIN_BUILTIN_LED, Pin.OUT, value=1)    
pwm0 = PWM(Pin(PIN_Servo))      

p0.on() 
p2.off()     
pwm0.freq(50) 
i = 72 # off 
pwm0.duty(i)          
time.sleep(0.5)
i = 85 # idle
pwm0.duty(i)          
time.sleep(0.5)
i = 93 # on 
pwm0.duty(i)          
time.sleep(0.5)
pwm0.deinit()
'''

for cmd in a.splitlines():
    webrepl.send(cmd)
    webrepl.send('\r')
    print(webrepl.recv()) 


>>> 
import time  
>>> 
from machine import Pin, PWM 
>>> 
PIN_MOSFET = 4
>>> 
PIN_Servo = 5 
>>> 
PIN_BUILTIN_LED = 2
>>> 
p0 = Pin(PIN_MOSFET, Pin.OUT, value=0)  
>>> 
p2 = Pin(PIN_BUILTIN_LED, Pin.OUT, value=1)    
>>> 
pwm0 = PWM(Pin(PIN_Servo))      
>>> 

>>> 
p0.on() 
>>> 
p2.off()     
>>> 
pwm0.freq(50) 
>>> 
i = 72 # off 
>>> 
pwm0.duty(i)          
>>> 
time.sleep(0.5)
>>> 
i = 85 # idle
>>> 
pwm0.duty(i)          
>>> 
time.sleep(0.5)
>>> 
i = 93 # on 
>>> 
#pwm0.duty(i)          
>>> 
time.sleep(0.5)
>>> 
pwm0.deinit()
>>> 


In [4]:
webrepl.close()